In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.functional import mse_loss


In [ ]:
class DGM_Layer(nn.Module):
    
    def __init__(self, dim_x, dim_S, activation='Tanh'):
        super(DGM_Layer, self).__init__()
        
        if activation == 'ReLU':
            self.activation = nn.ReLU()
        elif activation == 'Tanh':
            self.activation = nn.Tanh()
        elif activation == 'Sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'LogSigmoid':
            self.activation = nn.LogSigmoid()
        else:
            raise ValueError("Unknown activation function {}".format(activation))
            

        self.gate_Z = self.layer(dim_x+dim_S, dim_S)
        self.gate_G = self.layer(dim_x+dim_S, dim_S)
        self.gate_R = self.layer(dim_x+dim_S, dim_S)
        self.gate_H = self.layer(dim_x+dim_S, dim_S)
            
    def layer(self, nIn, nOut):
        l = nn.Sequential(nn.Linear(nIn, nOut), self.activation)
        return l
    
    def forward(self, x, S):
        x_S = torch.cat([x,S],1)
        Z = self.gate_Z(x_S)
        G = self.gate_G(x_S)
        R = self.gate_R(x_S)
        
        input_gate_H = torch.cat([x, S*R],1)
        H = self.gate_H(input_gate_H)
        
        output = ((1-G))*H + Z*S
        return output
class Net_DGM(nn.Module):

    def __init__(self, dim_x, dim_S, activation='Tanh'):
        super(Net_DGM, self).__init__()

        self.dim = dim_x
        if activation == 'ReLU':
            self.activation = nn.ReLU()
        elif activation == 'Tanh':
            self.activation = nn.Tanh()
        elif activation == 'Sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'LogSigmoid':
            self.activation = nn.LogSigmoid()
        else:
            raise ValueError("Unknown activation function {}".format(activation))

        self.input_layer = nn.Sequential(nn.Linear(dim_x+1, dim_S), self.activation)

        self.DGM1 = DGM_Layer(dim_x=dim_x+1, dim_S=dim_S, activation=activation)
        self.DGM2 = DGM_Layer(dim_x=dim_x+1, dim_S=dim_S, activation=activation)
        self.DGM3 = DGM_Layer(dim_x=dim_x+1, dim_S=dim_S, activation=activation)

        self.output_layer = nn.Linear(dim_S, 1)

    def forward(self,t,x):
        tx = torch.cat([t,x], 1)
        S1 = self.input_layer(tx)
        S2 = self.DGM1(tx,S1)
        S3 = self.DGM2(tx,S2)
        S4 = self.DGM3(tx,S3)
        output = self.output_layer(S4)
        return output

In [ ]:
dim_x = 2  # 1 dimension for time + 2 dimensions for space
dim_S = 100  # Assume we choose the hidden layer size as 100
activation = 'Tanh'  # Choose 'Tanh' as the activation function

# Create an instance of the Net_DGM model
u = Net_DGM(dim_x=dim_x, dim_S=dim_S, activation=activation)

In [ ]:
T = 1.0  # Upper bound of the time interval
N = 2000  # Number of sampling points

# Randomly generate time points t ∈ [0, T)
t_samples = torch.rand(N, 1) * T

# Randomly generate spatial points x ∈ [-1, 1] x [-1, 1]
x_samples = (torch.rand(N, 2) * 20) - 10

In [ ]:
sigma = torch.tensor([[0.001], [0.001]], dtype=torch.float32, requires_grad=False)

H = torch.tensor([[0.1, 0.0],
                  [0.0, 0.1]], dtype=torch.float32, requires_grad=False)

M = torch.tensor([[1, 0],
                  [0, 1]], dtype=torch.float32, requires_grad=False)

C = torch.tensor([[0.0, 0.0],
                  [0.0, 0.0]], dtype=torch.float32, requires_grad=False)

D = torch.tensor([[1, 0],
                  [0, 1]], dtype=torch.float32, requires_grad=False)

R = torch.tensor([[10.0, 0.0],
                  [0.0, 10.0]], dtype=torch.float32, requires_grad=False)

a = torch.tensor([1.0, 1.0], dtype=torch.float32, requires_grad=False)

In [ ]:
def compute_total_loss(model, t_samples, x_samples, sigma, H, M, C, D, R, alpha):
    """
    Compute the total loss for the PDE problem, including the equation part and the boundary condition part.
    """
    # Ensure input requires gradients
    t_samples.requires_grad_(True)
    x_samples.requires_grad_(True)

    # model prediction
    u = model(t_samples, x_samples)
    print(type(u))
    # # Time derivative
    grad_u_t = torch.autograd.grad(u.sum(), t_samples, create_graph=True)[0]
    
    # Spatial derivative
    grad_u_x = torch.autograd.grad(u.sum(), x_samples, create_graph=True, retain_graph=True)[0]
    # Compute second derivatives for each dimension
    hessians = []
    for i in range(t_samples.size(0)):
        def u_function(x_samples):
            #Expand t and x to at least one-dimensional tensors to enable concatenation
            t_expanded = t_samples[i].unsqueeze(0)
            x_expanded = x_samples
            return model(t_expanded, x_expanded).sum()

        grad_u_xx = torch.autograd.functional.hessian(u_function, x_samples[i].unsqueeze(0))
        hessians.append(grad_u_xx)

    # use torch.squeeze()extra dimension，and collect processed tensors
    squeezed_hessians = [torch.squeeze(tensor) for tensor in hessians]
    
    # Stack processed tensors using torch.stack()
    stacked_hessians = torch.stack(squeezed_hessians)
    grad_u_xx = stacked_hessians
    
    # calculate σσ^T
    sigma_sigma_T = sigma @ sigma.T
    # Assume n is the size of the first dimension of stacked_hessians
    n = stacked_hessians.size(0)

    # Add a new first dimension to sigma_sigma_T to make its shape [1, 2, 2]
    sigma_sigma_T_unsqueezed = sigma_sigma_T.unsqueeze(0)

    # Expand the size of the first dimension to n using the expand method to make its shape [n, 2, 2]
    expanded_sigma_sigma_T = sigma_sigma_T_unsqueezed.expand(n, -1, -1)
    print(expanded_sigma_sigma_T.size(),grad_u_xx.size())
    product = torch.matmul(expanded_sigma_sigma_T, grad_u_xx)

    # Compute the sum of each matrix diagonal element, i.e., trace, the result's shape is [n, 1]
    trace = torch.diagonal(product, dim1=-2, dim2=-1).sum(-1, keepdim=True)

    # Final diffusive term, multiply by 0.5
    diffusive_term = 0.5 * trace
    print(diffusive_term.size)
    convective_term = torch.sum(grad_u_x * (H @ x_samples.T).T, dim=1, keepdim=True) + torch.sum(grad_u_x * (M @ alpha.unsqueeze(1)).T, dim=1, keepdim=True)
    quadratic_term = torch.sum(x_samples * (C @ x_samples.T).T, dim=1, keepdim=True)
    constant_term = alpha @ D @ alpha
    print(grad_u_t.size(),diffusive_term.size(),convective_term.size(),quadratic_term.size())
    pde_residual = grad_u_t + diffusive_term + convective_term + quadratic_term + constant_term
    
    # Loss for the equation part
    loss_eqn = pde_residual.pow(2).mean()
    
    # Loss for the boundary condition part
    T = torch.tensor(1.0, requires_grad=False).expand_as(t_samples)  # 假设边界在T=1时
    u_T = model(T, x_samples)
    boundary_condition = (u_T - torch.sum(x_samples * (R @ x_samples.T).T, dim=1, keepdim=True)).pow(2)
    loss_boundary = boundary_condition.mean()
    
    # Total loss
    total_loss = loss_eqn + loss_boundary
    
    return total_loss

In [ ]:
#%% MC simulation
class LQR:
    def __init__(self, H, M, C, D, R, T,sigma):
        self.H = torch.tensor(H, dtype=torch.float32)
        self.M = torch.tensor(M, dtype=torch.float32)
        self.C = torch.tensor(C, dtype=torch.float32)
        self.D = torch.tensor(D, dtype=torch.float32)
        self.R = torch.tensor(R, dtype=torch.float32)
        self.T = T
        self.sigma = sigma 

    def solve_ricatti_ode(self, time_grid):
        H, M, C, D, R = self.H.numpy(), self.M.numpy(), self.C.numpy(), self.D.numpy(), self.R.numpy()
        T = self.T

        def ricatti_ode(t, S_flat):
            S = S_flat.reshape(2, 2)
            dSdt = -2 * H.T @ S + S @ M @ np.linalg.inv(D) @ M.T @ S - C
            return dSdt.flatten()
        sol = solve_ivp(ricatti_ode, [T, 0], R.flatten(), t_eval=np.flip(time_grid), vectorized=True,rtol=1e-6, atol=1e-9)
        S_values = sol.y.T.reshape(-1, 2, 2)
        S_values_tensor = torch.tensor(S_values, dtype=torch.float32)
        S_values_tensor_reversed = torch.flip(S_values_tensor, dims=[0])
        return S_values_tensor_reversed
    
    def control_problem_value(self, t, x):
        S_values = self.solve_ricatti_ode(t)
        v_values = torch.zeros(x.size(0), 1, dtype=torch.float32)  # initialize v_values
        # for cycle
        for i in range(x.size(0)):
            x_i = x[i]  # x_i shape as (1, 2)
            S_i = S_values[i]  # S_i shape (2, 2)
            v_i = torch.matmul(torch.matmul(x_i.transpose(0, 1), S_i), x_i)  # calculate x_i * S_i * x_i'

            integral_value = 0  # initialize integral
            if self.sigma is not None:  
                sigma_mat = torch.tensor(self.sigma, dtype=torch.float32)
                
                for j in range(i, len(t) - 1):
                    S_j = S_values[j]  
                    tr_value = torch.trace(torch.matmul(torch.matmul(sigma_mat, sigma_mat.transpose(0, 1)), S_j))
                    delta_t = t[j + 1] - t[j]  
                    integral_value += tr_value * delta_t  

            v_i += integral_value  
            v_values[i] = v_i
        return v_values
    def markov_control_function(self, t, x):
        S_values = self.solve_ricatti_ode(t)  
       
        a_values = [-torch.inverse(self.D) @ self.M.T @ S_values[i].clone().detach().to(dtype=torch.float32) @ x_[None, :] for i, x_ in enumerate(x)]
        
        a_values = torch.stack(a_values).squeeze(1)  
        return a_values

    
def wiener_process(N,t,T):              
    dW = np.random.normal(0, np.sqrt((T-t)/N))
    return dW


def simulation (lqr,x,t,T,N):
    '''
    this function simulate single path

    '''
    time_grid = torch.linspace(t, T, steps=N+1)
    dt = (T-t)/N
    x_values = torch.zeros((N+1, 2, 1))
    x_values[0] = x
    a = np.array([[1], [1]])
    for n in range(N):
        x_n = x_values[n]

        dW =  wiener_process(N,t,T) 
        sigma_dW =  lqr.sigma * dW
        x_next = x_n + dt * (lqr.H @ x_n +lqr.M.T@ a) + sigma_dW
        x_values[n+1] = x_next
    return x_values

def simulate_multiple_paths(lqr, x,t, T, N, n):
    '''
    
    t:initial time
    
    '''
    
    all_simulations = []

    for _ in range(n):
        simulation_result = simulation(lqr, x,t, T, N)
        all_simulations.append(simulation_result)

    
    stacked_simulations = torch.stack(all_simulations)
    
    return stacked_simulations

#%%
def compute_J_alpha(lqr, X_paths,t,T, N,n):
    C = lqr.C
    D = lqr.D
    R = lqr.R
    dt = (T-t)/N
    a = np.array([[1], [1]])
    
    a_tensor = torch.tensor(a, dtype=torch.float32)
    a_expanded = a_tensor.unsqueeze(0).unsqueeze(0)  
    a_expanded = a_expanded.expand(n, N+1, -1, -1)  
    
    print(a_expanded.size(),X_paths.size())
    integral_part = torch.sum(X_paths.transpose(2, 3) @ C @ X_paths + a_expanded.transpose(2, 3) @ D @ a_expanded, dim=1) * dt
    
    terminal_cost = torch.sum(X_paths[:, -1].transpose(1, 2) @ R @ X_paths[:, -1], dim=1)

    
    J_alpha = torch.mean(integral_part + terminal_cost)
    return J_alpha.item()
#%%
H = [[0.1, 0.0],
     [0.0, 0.1]]
M = [[1, 0],
     [0, 1]]
C = [[0.0, 0.0],
     [0.0, 0.0]]
D = [[1, 0],
     [0, 1]]
R = [[10.0, 0.0],
     [0.0, 10.0]]
T = 1
t=0
sigma =np.array([[0.001], [0.001]])
N=100
time_grid = np.linspace(t, T,N+1)

initial_x = torch.tensor([[[0.0], [0.0]]], dtype=torch.float32)
lqr = LQR(H=H, M=M, C=C, D=D, R=R, T=T,sigma=sigma)
#%%
n_simulate = simulate_multiple_paths(lqr, initial_x,t, T, N, n=100)
J = compute_J_alpha(lqr, n_simulate,t,T, N,n=100)
#%%
# fix random seed to makesure same output
np.random.seed(0)
torch.manual_seed(0)

# uniformly distributed (0,1)
t_values = np.linspace(0, 1, 5)

#Generate 100 sets of x values uniformly distributed, where each set of x values is within the range [-10, 10] x [-10, 10].
x_values = np.random.uniform(-10, 10, (25, 2))
x_values_all = []
t_values_all = []
J_values = []

# for each t, use all x to calculate J
for t in t_values:
    for x in x_values:
        n_simulate = simulate_multiple_paths(lqr, torch.tensor(x, dtype=torch.float32).reshape(1, 2, 1),t, T, N, n=100)
        J = compute_J_alpha(lqr, n_simulate,t,T, N,n=100)
        t_values_all.append(t)
        x_values_all.append(x)
        J_values.append(J)

In [ ]:
num_epochs = 800  
loss_all = []
mse_all = []
optimizer = torch.optim.Adam(u.parameters(), lr=0.008) 
for epoch in range(num_epochs):
    optimizer.zero_grad()  
    loss = compute_total_loss(u, t_samples, x_samples, sigma, H, M, C, D, R, a)  # Compute loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update parameters
    loss_all.append(loss)
    # Print loss every 10 epochs
    if (epoch + 1) % 1 == 0:
        J_dgm = []
        for t, x in zip(t_values_all, x_values_all):
            # Assuming  model u takes time t and state x as input
            #  Depending on the implementation details of  model u, we need to adjust the shape of the input here
            x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
            t_tensor = torch.tensor([[t]], dtype=torch.float32)
            J_pred = u(x_tensor, t_tensor)
            J_dgm.append(J_pred.item())
        mse_value = mse_loss(torch.tensor(J_dgm), torch.tensor(J_values))
        mse_all.append(mse_value.item())


In [ ]:
loss_values = [loss.item() for loss in loss_all]
plt.loglog(range(1, len(loss_values) + 1), loss_values)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Log-Log Plot of Loss')
plt.grid(True) 
plt.show()

In [ ]:
mse_all
plt.loglog(range(1, len(mse_all) + 1), mse_all)
plt.xlabel('Epoch')
plt.ylabel('Mse')
plt.title('Log-Log Plot of Loss')
plt.grid(True) 
plt.show()